In [19]:
import sympy
from sympy.physics.control.lti import TransferFunction
import functools 

# Planteo del modelo
## Ecuaciones
Sean 

$\rarr$ x1: temperatura actual del sistema

$\rarr$ x2: Temperatura del resistor



In [15]:
from sympy.abc import s # Laplace var 
a,b,c,d,e= sympy.symbols("a,b,c,d,e") # State space var 

A = sympy.Matrix([[a,b],[c,d]])
B = sympy.Matrix([[0],[e]])
C = sympy.Matrix([1,0]).T

### Matriz dinámica

In [7]:
A

Matrix([
[a, b],
[c, d]])

### Matriz de control

In [8]:
B

Matrix([
[0],
[e]])

### Matriz de salida

In [9]:
C

Matrix([[1, 0]])

## Funcion de transferencia

El calculo de la funcion de transferencia se hace a través de la formula:


$$
 H(s) = C{(sI - A)^{-1}}B + D
$$


In [17]:
temp = s * sympy.eye(A.shape[0]) - A # sI - A
inv = temp.inv()                  # (sI - A)^-1
temp = C * inv                  # C * (sI - A)^-1
H = temp * B                   # C * (sI - A)^-1 * B
transfer_function = H[0]      # Extract the result

n,d = sympy.fraction(sympy.factor(transfer_function)) # Factorize (factor) and extract numerator and denominator (fraction)
tf_expr = n/d   ##This is useful for jordan form

tf = TransferFunction(n,d,s)    # Create a transfer function object, which can be used for further analysis
tf

TransferFunction(b*e, a*d - a*s - b*c - d*s + s**2, s)

## Formas canonicas

Para el calculo de las formas canonicas, primero se preparan los datos para su facil manipulacion luego

In [24]:
den_args_str = [str(x) for x in d.args] # Extract terms from denominator
pow2 = [x for x in den_args_str if "s**2" in x] # Terms that have s ^ 2
pow1 = [x for x in den_args_str if "*s" in x and x not in pow2] # Terms that have s ^ 1
tail_terms = [x for x in den_args_str if x not in pow1 and x not in pow2] # Terms without s


pow1_expr =sympy.factor(sympy.parse_expr(functools.reduce(lambda a,b: a + b, pow1), evaluate=False)) # Join the terms
tail_terms_expr = sympy.factor(sympy.parse_expr(functools.reduce(lambda a,b: a + b, tail_terms), evaluate=False)) # Join the terms


def sanitize_expr_str(e):
    """
    Delete the s^2 and s^1 from the expression
    """
    start = e.find("s")
    if start == -1:
        return e 
    return e.replace(e[start:start+2:1], "")

# Find the coefficients a_0 and a_1
a_0 = sympy.parse_expr(sanitize_expr_str(str(tail_terms_expr)), evaluate=False)
a_1 =sympy.parse_expr(sanitize_expr_str(str(pow1_expr)), evaluate=False)



### Forma de control

In [21]:
A_ctrl = sympy.Matrix([[0,1], [a_0, a_1]])
B_ctrl = sympy.Matrix([[0], [1]])
C_ctrl = sympy.Matrix([n,0]).T

In [22]:
A_ctrl

Matrix([
[        0,      1],
[a*d - b*c, -a - d]])

In [23]:
B_ctrl

Matrix([
[0],
[1]])

In [24]:
C_ctrl

Matrix([[b, 0]])

## Forma observable
Transponemos el A de control e intercambiamos B y C de control

In [25]:

import copy

A_obs = copy.deepcopy(A_ctrl).T
B_obs = copy.deepcopy(C_ctrl)
C_obs = copy.deepcopy(B_ctrl).T


In [26]:
A_obs

Matrix([
[0, a*d - b*c],
[1,    -a - d]])

In [27]:
B_obs

Matrix([[b, 0]])

In [28]:
C_obs

Matrix([[0, 1]])

In [30]:
tf_poles = tf.poles()


[a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2,
 a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2]

In [31]:
tf_poles[0]


a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2

In [32]:
tf_poles[1]


a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2

## Forma de Jordan

In [34]:
## Para esta forma de jordan, vamos a asumir que hay todos polos reales y distintos 
## A la hora de definir valores para todos los parametros, podemos concluir
## si hace falta modificarlo 
A_jordan = sympy.Matrix([[tf_poles[0], 0], [0,tf_poles[1]]])
A_jordan

Matrix([
[a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2,                                               0],
[                                              0, a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2]])

In [54]:
B_jordan = sympy.ones(len(tf_poles),1)
B_jordan

Matrix([
[1],
[1]])

In [50]:
#residue calcula el residuo de las fracciones parciales para la funcion
#de transferencia
C_jordan = [sympy.residue(tf_expr, s, tf_poles[0]), sympy.residue(tf_expr, s, tf_poles[1])]
C_jordan

[-b/sqrt(a**2 - 2*a*d + 4*b*c + d**2), b/sqrt(a**2 - 2*a*d + 4*b*c + d**2)]